In [1]:
from gmath import *
from process import *
import os

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.9.13
[Taichi] Starting on arch=cuda


In [2]:
pixels = load_data()
c2w = load_mat()
rays = ti.Vector.field(3,ti.f32, shape=(len(pixels), pixels[0].shape[0], pixels[0].shape[1]))
origins = ti.Vector.field(3,ti.f32, shape=(len(pixels)))
print(len(pixels),c2w.shape,rays.shape,origins.shape)

Loading Hotdog/rgb/


 25%|██▌       | 100/400 [00:09<00:28, 10.57it/s]

100 images loaded successfully with the shape of (800, 800)
Loading Hotdog/pose/



 25%|██▌       | 100/400 [00:00<00:00, 4763.39it/s]


100 matrices loaded successfully
100 (100,) (100, 800, 800) (100,)


In [3]:
for i in range(len(pixels)):
    calRays(i,rays,c2w,origins)

In [4]:
print(origins[0])
print(rays[0,0,0],rays[0,799,799],rays[0,399,399])

[ -9.36606884 395.22384644 -84.36512756]
[-0.33442166 -0.93625021 -0.10769226] [ 0.37207747 -0.81354672  0.44687825] [ 0.02062614 -0.98159271  0.18986891]


In [5]:
vs = voxel_system()
vs.zero_init()
# vs.random_init()

In [6]:
buffer=ti.Vector.field(3,dtype=ti.f32,shape=(800,800))

In [7]:
for i in tqdm(range(100)):
    
    k=i#np.random.randint(0,100)
    # renderBuffer(buffer,vs.content,origins,rays,k)
    # renderBuffer_back(buffer,vs.content,origins,rays,k,pixels[k],vs.grad)
    cluster(buffer,origins,rays,k,pixels[k],vs.content)
    # vs.update(True)
    # vs.zero_grad()
    # vs.lr[None]*=0.98

100%|██████████| 100/100 [00:17<00:00,  5.77it/s]


In [8]:
vs.lr[None]=0.02
for i in tqdm(range(0,10)):
    for j in range(5):
        k=i
        renderBuffer(buffer,vs.content,origins,rays,k)
        renderBuffer_back(buffer,vs.content,origins,rays,k,pixels[k],vs.grad)
        vs.update(False)
        vs.zero_grad()
        

100%|██████████| 10/10 [01:52<00:00, 11.27s/it]


In [9]:
for i in tqdm(range(100)):
    
    k=i#np.random.randint(0,100)
    # renderBuffer(buffer,vs.content,origins,rays,k)
    # renderBuffer_back(buffer,vs.content,origins,rays,k,pixels[k],vs.grad)
    cluster(buffer,origins,rays,k,pixels[k],vs.content)
    # vs.update(True)
    # vs.zero_grad()
    # vs.lr[None]*=0.98

100%|██████████| 100/100 [00:03<00:00, 27.14it/s]


In [13]:
vs.lr[None]=0.003
for i in tqdm(range(20)):
    for j in range(10):
        k=np.random.randint(0,100)
        renderBuffer(buffer,vs.content,origins,rays,k)
        renderBuffer_back(buffer,vs.content,origins,rays,k,pixels[k],vs.grad)
    vs.update(True)
    vs.zero_grad()
        

100%|██████████| 20/20 [07:07<00:00, 21.36s/it]


In [20]:
gui = ti.GUI('window', (800,800), fast_gui=False)
while gui.running:
    gui.set_image(buffer)
    gui.show()

In [11]:
camera = Camera(vs)
while camera.gui.running:
        camera.cal_rays()
        camera.render()
        camera.display()
        camera.processEvents()

In [10]:
print(vs.content[150,150,150])
print(vs.content[140,140,140])
print(vs.content[299,299,299])
print(buffer[400,400])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0.]


RuntimeError: [taichi/rhi/cuda/cuda_driver.h:taichi::lang::CUDADriverFunction<void *>::operator ()@92] CUDA Error CUDA_ERROR_ASSERT: device-side assert triggered while calling stream_synchronize (cuStreamSynchronize)